Нам предстоит исследовать базу данных книг, у нас есть подробное описание данных и техническое задание. Что же приступаем

# SQL исследование базы данных книг

Загрузим библиотеки и получим доступ к SQL базе данных

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

сделаем функцию

In [3]:
def show (query):
    return pd.io.sql.read_sql(query, con = engine)

Протестим

In [4]:
show('''
            SELECT *
            FROM books 
            LIMIT 10
        ''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


Работает, едем дальше

### Задание 1. Посчитаем количество книг, выпущенных после 1 января 2000 года;

Есть таблица book, в которой есть столбцы book_id, author_id, title, num_pages, publication_date, publisher_id. Как раз есть дата публикация, отлично. Поставим условие по ней и посчитаем количество

In [5]:
show('''
            SELECT COUNT(*)
            FROM books 
            WHERE publication_date >= '2000-01-01'
        ''')

,count
0,821


821 книга была выпущена после 1 января 2000 года

### Задание 2. Посчитаем количество пользовательских обзоров и среднюю оценку для каждой книги;

Пользовательские обзоры у нас в reviews, оценки в ratings, книги в books. Кол-во обзоров и рейтов разное, придется делать 2 разных запроса сохранить и объединить их по книгам

In [6]:
show('''
    SELECT rate.title,
           rate.avg,
           rev.count
    FROM      (SELECT books.title,
               books.book_id,
               AVG (ratings.rating)
               FROM books
               LEFT JOIN ratings ON ratings.book_id = books.book_id
               GROUP BY books.title, books.book_id) as rate
    LEFT JOIN 
                (SELECT books.book_id,
                books.title,
                COUNT(review_id)
                FROM books
                LEFT JOIN reviews ON reviews.book_id = books.book_id
                GROUP BY books.book_id,
                books.title) as rev ON rev.book_id = rate.book_id
    ORDER BY rev.count DESC
        ''')

,title,avg,count
0,Twilight (Twilight #1),3.662500,7
1,Water for Elephants,3.977273,6
2,The Glass Castle,4.206897,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
4,The Curious Incident of the Dog in the Night-Time,4.081081,6
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,0
996,The Natural Way to Draw,3.000000,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
998,Essential Tales and Poems,4.000000,0


Получилась таблица с рейтингами и кол-вом обзоров... погодите Сумерки самый обозреваемый? гарри только на 4. Надеюсь это не настоящие данные, а учебные

### Задание 3. Определим издательство, которое издало наибольшее число книг толще 50 страниц

Книги в books, издательство в publishers. Объеденим и посмотрим

In [7]:
show('''
        SELECT publishers.publisher,
               COUNT(*)
        FROM books 
        LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
        WHERE num_pages > 50
        GROUP BY publishers.publisher
        ORDER BY COUNT(*) DESC
        LIMIT 1
        ''')

,publisher,count
0,Penguin Books,42


Вот и победитель! Сэр Аллен Лейном и его братья Ричард и Джон были бы довольны

### Задание 4. Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более пользовательскими оценками;

Придется объединять 3 таблицы книги, авторы, рейтинги. Причем сначала получим запрос по книгам с рейтингами, выберем только те, на которые было более 50 отзывов, потом объединим с таблицей авторов и посмотрим на топ 1 автора

In [9]:
show('''
            SELECT  author,
                    SUM(rate_books.count),
                    AVG(rate_books.avg)
            FROM authors
            LEFT JOIN
                             (SELECT books.author_id, 
                                    books.title,
                                    COUNT (ratings.rating_id),
                                    AVG (rating)
                             FROM books
                             LEFT JOIN ratings ON ratings.book_id = books.book_id
                             GROUP BY books.author_id, books.title
                             HAVING COUNT (ratings.rating_id) > 50) as rate_books on rate_books.author_id = authors.author_id
            WHERE rate_books.count > 1
            GROUP BY author 
            ORDER BY avg DESC
            LIMIT 1
        ''')

,author,sum,avg
0,J.K. Rowling/Mary GrandPré,310.0,4.283844


Весьма заслуженно! Отлично

### Задание 5. Посчитаем среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

Тут требуется 2 таблицы ratings и reviews

In [10]:
show('''
     SELECT AVG (count_revievs)
     
     FROM   (SELECT ratings.username,
               COUNT (DISTINCT ratings.rating_id) as count_ratings,
               COUNT (DISTINCT reviews.review_id) as  count_revievs
        FROM  ratings
        LEFT JOIN reviews  ON reviews.username = ratings.username
        GROUP BY ratings.username
        HAVING COUNT(DISTINCT ratings.rating_id) > 50
        ORDER BY count_ratings DESC) as sub
        ''')

,avg
0,24.333333


Среднее количество текстовых обзоров пользователей, которые поставили более чем 50 оценок 24.33

Спасибо!